In [54]:
import geopandas as gpd
from shapely import geometry
import numpy as np
import urllib
import cv2
import rasterio
import os
from os.path import exists

import matplotlib.pyplot as plt
from multiprocessing import Pool
import mercantile
from tenacity import retry, stop_after_attempt


In [42]:
geojson_path = '../us-county-boundaries.geojson'
gdf = gpd.read_file(geojson_path, crs=4326)
gdf.head()

,intptlat,countyfp_nozero,countyns,stusab,csafp,state_name,aland,geoid,namelsad,countyfp,awater,classfp,lsad,name,funcstat,cbsafp,intptlon,statefp,mtfcc,geometry
0,+41.8060534,3,00212338,CT,278,Connecticut,1903543898,09003,Hartford County,003,40543777,H4,06,Hartford,N,25540,-072.7329157,09,G4020,"POLYGON ((-72.94902 41.80643, -72.94894 41.806..."


In [27]:
bounds = gdf.geometry.loc[0].bounds
bounds

(-73.029537, 41.544725, -72.407874, 42.038785)

In [56]:
@retry(stop=stop_after_attempt(3))
def extractSatelliteImages(minX,minY,maxX,maxY,height='512',width='512'):

    url = f"http://wms3.mapsavvy.com/WMSService.svc/db45ac1c32ac4e9caa5ecc3473998c81/WMSLatLon?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&LAYERS=Aerial&SRS=EPSG:4326&CRS=EPSG:4326&BBOX={minX},{minY},{maxX},{maxY}&WIDTH={width}&HEIGHT={height}&STYLES=&TRANSPARENT=false&FORMAT=image/png"
    req = urllib.request.Request(url)
    resp = urllib.request.urlopen(req)
    arr = np.asarray(bytearray(resp.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def convertToTiff(img, minlat,minlong,maxlat,maxlong,height=512,width=512,imagename='sample_image',path=None, crs="epsg:4326"):
    filename = imagename + '.tiff'

    transform = rasterio.transform.from_bounds(minlong, minlat, maxlong, maxlat, width, height)

    with rasterio.open(os.path.join(path, filename), 'w', driver='GTiff', dtype=rasterio.uint8, count=3, width=width, height=height, transform=transform, crs=crs) as dst:
        for index in range(3):
            dst.write(img[:,:,index], indexes=index + 1)


tiles = mercantile.tiles(bounds[0],bounds[1],bounds[2],bounds[3],zooms=17)
for t in tiles :
    val = mercantile.bounds(t)
    print(val.west,val.south,val.east,val.north)
    img = extractSatelliteImages(val.west,val.south,val.east,val.north)
    img = convertToTiff(img,val.west,val.south,val.east,val.north,path='../data')
    break

-73.0316162109375 42.0370543018838 -73.02886962890625 42.039094188385945


In [15]:
for t in tiles:
    print(t)